In [1]:
import xib.cfg as cfg

In [2]:
import pdb

In [3]:
cfg.Index.get_feature(0)

<Ptype.CONSONANT: Index(g_idx=0, c_idx=0, f_idx=0)>

In [4]:
from pathlib import Path

In [5]:
path = Path('/scratch/j_luo/data/phones/phones_merged.tsv')

In [6]:
import pandas as pd

In [8]:
import ast

In [9]:
import inflection

In [13]:
converters = {cat.name.lower(): ast.literal_eval  for cat in cfg.Category}

In [14]:
df = pd.read_csv(path, sep='\t', converters=converters)

In [15]:
df.head()

,segment,lang,ipa,merged_ipa,ptype,c_voicing,c_place,c_manner,v_height,v_backness,v_roundness,diacritics,s_stress,s_length,s_break,t_level,t_contour,t_global
0,nɔːme,zza,"['n', 'ɔ', 'ː', 'm', 'e']","['n', 'ɔː', 'm', 'e']","[consonant, vowel, consonant, vowel]","[voiced, none, voiced, none]","[alveolar, none, bilabial, none]","[nasal, none, nasal, none]","[none, open-mid, none, close-mid]","[none, back, none, front]","[none, rounded, none, unrounded]","[none, none, none, none]","[none, none, none, none]","[none, long, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]"
1,ɾaŋɡ,zza,"['ɾ', 'a', 'ŋ', 'ɡ']","['ɾ', 'a', 'ŋ', 'ɡ']","[consonant, vowel, consonant, consonant]","[voiced, none, voiced, voiced]","[alveolar, none, velar, velar]","[flap, none, nasal, plosive]","[none, open, none, none]","[none, front, none, none]","[none, unrounded, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]"
2,pɔː,zza,"['p', 'ɔ', 'ː']","['p', 'ɔː']","[consonant, vowel]","[voiceless, none]","[bilabial, none]","[plosive, none]","[none, open-mid]","[none, back]","[none, rounded]","[none, none]","[none, none]","[none, long]","[none, none]","[none, none]","[none, none]","[none, none]"
3,bari,zza,"['b', 'a', 'r', 'i']","['b', 'a', 'r', 'i']","[consonant, vowel, consonant, vowel]","[voiced, none, voiced, none]","[bilabial, none, alveolar, none]","[plosive, none, trill, none]","[none, open, none, close]","[none, front, none, front]","[none, unrounded, none, unrounded]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]"
4,herb,zza,"['h', 'e', 'r', 'b']","['h', 'e', 'r', 'b']","[consonant, vowel, consonant, consonant]","[voiceless, none, voiced, voiced]","[glottal, none, alveolar, bilabial]","[non-sibilant-fricative, none, trill, plosive]","[none, close-mid, none, none]","[none, front, none, none]","[none, unrounded, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]","[none, none, none, none]"


In [21]:
from tqdm import tqdm
tqdm.pandas()

In [41]:
for feat in cfg.Category:
    col = feat.name.lower()
    new_col = f'{col}.upper()'
    cat_cls = getattr(cfg, inflection.camelize(feat.name.lower()))
    df[new_col] = df[col].progress_apply(lambda lst: [getattr(cat_cls, x.replace('-', '_').upper()).value.g_idx for x in lst])


100%|██████████| 453777/453777 [00:03<00:00, 135859.35it/s]


In [43]:
col_names = [f'{feat.name.lower()}_idx' for feat in cfg.Category]

In [46]:
smaller = df[['segment', 'lang', 'merged_ipa'] + col_names]

In [47]:
smaller.to_csv('/scratch/j_luo/data/phones/phones_merged_idx.tsv', sep='\t', index=False)

In [53]:
# Languages to keep.
from collections import Counter
cnt = Counter(smaller['lang'])

In [59]:
langs = set([lang for lang, count in sorted(cnt.items(), key=lambda x: x[1], reverse=True) if count >= 1000])

In [60]:
in_langs = smaller['lang'].progress_apply(lambda x: x in langs)

100%|██████████| 453777/453777 [00:00<00:00, 1025361.83it/s]


In [62]:
filtered = smaller[in_langs]

In [68]:
import torch
import numpy as np

In [75]:
for lang in langs:
    out_path = f'/scratch/j_luo/data/phones/phones_{lang}_idx.pth'
    tmp_df = filtered[filtered['lang'] == lang]
    segments = tmp_df['segment'].values
    matrices = list()
    for r, s in tqdm(tmp_df.iterrows(), total=len(tmp_df)):
        arr = np.stack([s[col] for col in col_names], axis=1)
        tensor = torch.from_numpy(arr)
        matrices.append(tensor)
    out = {
        'segments': segments,
        'matrices': matrices
    }
    torch.save(out, out_path)

100%|██████████| 1760/1760 [00:00<00:00, 3575.74it/s]


In [74]:
matrices[0].shape

torch.Size([6, 14])

In [80]:
next(iter(cfg.Category)).name

'PTYPE'